In [7]:
# tag::importst[]
import streamlit as st
# end::importst[]
# tag::importvector[]
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
# end::importvector[]
# tag::importqa[]
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
# end::importqa[]
# tag::importretrievalqa[]
from langchain.chains import RetrievalQA
# end::importretrievalqa[]

# This file is in the solutions folder to separate the solution
# from the starter project code.
from llm import llm, embeddings




# Tool 1: Vector Search Index

In [16]:
# tag::vector[]
neo4jvector = Neo4jVector.from_existing_index(
    embeddings,                              # <1>
    url="bolt://18.207.204.78:7687",             # <2>
    username="neo4j",   # <3>
    password="gas-attractions-typists",   # <4>
    index_name="moviePlots",                 # <5>
    node_label="Movie",                      # <6>
    text_node_property="plot",               # <7>
    embedding_node_property="plotEmbedding", # <8>
    retrieval_query="""
RETURN
    node.plot AS text,
    score,
    {
        title: node.title,
        directors: [ (person)-[:DIRECTED]->(node) | person.name ],
        actors: [ (person)-[r:ACTED_IN]->(node) | [person.name, r.role] ],
        tmdbId: node.tmdbId,
        source: 'https://www.themoviedb.org/movie/'+ node.tmdbId
    } AS metadata
"""
)
# end::vector[]

# tag::retriever[]
retriever = neo4jvector.as_retriever()
# end::retriever[]

# tag::qa[]
kg_qa = RetrievalQA.from_chain_type(
    llm,                  # <1>
    chain_type="stuff",   # <2>
    retriever=retriever,  # <3>
    verbose=True          # <4>
)
# end::qa[]

# tag::generate-response[]
def generate_response(prompt):
    """
    Use the Neo4j Vector Search Index
    to augment the response from the LLM
    """

    # Handle the response
    response = kg_qa.invoke({"query": prompt}, verbose=True)

    return response['result']
# end::generate-response[]

In [17]:
response = generate_response("Which movies have similar plot as Toy Story?")

print(response)



> Entering new RetrievalQA chain...

> Finished chain.
Some movies with similar themes to Toy Story are "The Lego Movie," "Wreck-It Ralph," and "The Secret Life of Pets."


## Tool 2 Cyper Query

In [4]:
from langchain.chains import GraphCypherQAChain

from llm import llm
from graph import graph

cypher_qa = GraphCypherQAChain.from_llm(
    llm,          # (1)
    graph=graph,  # (2)
    verbose=True  # (3)
)

In [6]:
cypher_qa.invoke("Who played in Toy Story?")["result"]



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Toy Story"})<-[:ACTED_IN]-(a:Actor)
RETURN a.name
Full Context:
[{'a.name': 'Jim Varney'}, {'a.name': 'Tim Allen'}, {'a.name': 'Tom Hanks'}, {'a.name': 'Don Rickles'}]

> Finished chain.


'Jim Varney, Tim Allen, Tom Hanks, and Don Rickles played in Toy Story.'